# Cassava Leaf Disease Classification
Details: [kaggle.com](https://www.kaggle.com/c/cassava-leaf-disease-classification)

## Prepare kaggle dataset

In [1]:
!pip install -q kaggle
!wget -q https://github.com/MohammadRaziei/Deep-Learning-Course/raw/master/Project/kaggle/key/kaggle.json -O ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets list

ref                                                    title                                                size  lastUpdated          downloadCount  
-----------------------------------------------------  --------------------------------------------------  -----  -------------------  -------------  
gpreda/covid-world-vaccination-progress                COVID-19 World Vaccination Progress                  41KB  2021-02-01 08:48:19           5222  
ayushggarg/all-trumps-twitter-insults-20152021         All Trump's Twitter insults (2015-2021)             581KB  2021-01-20 16:51:05           1054  
sevgisarac/temperature-change                          Temperature change                                  778KB  2020-12-24 20:06:36           1146  
jorgesandoval/wind-power-generation                    Wind Power Generation Data                          245KB  2021-01-07 18:45:08            459  
cdminix/us-drought-meteorological-data                 US Drought & Meteorological Data       

In [24]:
!mkdir -p Dataset
!kaggle competitions download -c cassava-leaf-disease-classification --path Dataset

  0% 0.00/141k [00:00<?, ?B/s]
100% 141k/141k [00:00<00:00, 40.2MB/s]
 97% 211M/217M [00:08<00:00, 17.2MB/s]
100% 217M/217M [00:08<00:00, 25.7MB/s]
 98% 214M/218M [00:08<00:00, 29.7MB/s]
100% 218M/218M [00:09<00:00, 23.3MB/s]
 95% 205M/216M [00:01<00:00, 152MB/s]
100% 216M/216M [00:01<00:00, 156MB/s]
 90% 196M/217M [00:01<00:00, 147MB/s]
100% 217M/217M [00:01<00:00, 154MB/s]
 92% 198M/216M [00:01<00:00, 119MB/s] 
100% 216M/216M [00:01<00:00, 128MB/s]
 96% 209M/217M [00:09<00:00, 17.9MB/s]
100% 217M/217M [00:09<00:00, 23.8MB/s]
 96% 205M/214M [00:09<00:00, 15.8MB/s]
100% 214M/214M [00:09<00:00, 24.1MB/s]
 94% 202M/216M [00:01<00:00, 117MB/s] 
100% 216M/216M [00:01<00:00, 124MB/s]
 96% 209M/217M [00:08<00:00, 20.8MB/s]
100% 217M/217M [00:08<00:00, 27.4MB/s]
 99% 214M/217M [00:08<00:00, 29.0MB/s]
100% 217M/217M [00:08<00:00, 26.2MB/s]
 97% 209M/216M [00:08<00:00, 16.8MB/s]
100% 216M/216M [00:08<00:00, 25.4MB/s]
 91% 196M/216M [00:01<00:00, 96.5MB/s]
100% 216M/216M [00:01<00:00, 115MB/s] 


In [ ]:
files= glob.glob('./*zip')+ glob.glob('./*.jpg')+glob.glob('./*.csv')
remove_path()

In [17]:
import zipfile
import shutil, pathlib
from tqdm.auto import tqdm

def unzip_file(filename, target_path='.'):
  zf = zipfile.ZipFile(filename, 'r')
  zf.extractall(target_path)
  zf.close()

def unzip_files(filename_list, target_path='.'):
  for filename in tqdm(zip_files):
    unzip_file(filename, target_path)

def create_path(path):
  pathlib.Path(path).mkdir(parents=True, exist_ok=True)

def remove_path(path):
  shutil.rmtree(path, ignore_errors=True)

def recreate_path(path):
  remove_path(path)
  create_path(path)

In [23]:
import glob
zip_files = glob.glob('Dataset/*.zip')
unzip_files(zip_files, target_path='Dataset')

In [5]:
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from tqdm import tqdm
from PIL import Image


def resize_image(image, target_size=(100,100)):
    img = Image.fromarray(image)
    img = img.resize(target_size, Image.ANTIALIAS)
    img = np.asarray(img)
    return img

""" 
I recommend to see following websites:
  * https://stepup.ai/exploring_data_augmentation_keras/
  * https://machinelearningmastery.com/how-to-load-large-datasets-from-directories-for-deep-learning-with-keras/
"""
# create generator
datagen = ImageDataGenerator()
train_it = datagen.flow_from_directory('data/train/', class_mode='binary')
val_it = datagen.flow_from_directory('data/validation/', class_mode='binary')
test_it = datagen.flow_from_directory('data/test/', class_mode='binary')



In [ ]:
tf.data.TFRecordDataset(filenames = ['ld_train00-1338.tfrec.zip'])